In [ ]:
# import ipy_autoreload
%reload_ext autoreload
%autoreload 2
%aimport isochrone
from isochrone import *

import os
import geopandas as gpd
import numpy as np
import pandas as pd
import osmnx as ox
import networkx as nx
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point, LineString, Polygon
from shapely.prepared import prep
import alphashape
import cartopy.crs as ccrs

import lonboard
from lonboard.colormap import apply_categorical_cmap

In [ ]:
city = 'Cambridge'

In [ ]:
gdf_nodes, all_lts, G_lts = load_files(city)

In [ ]:
G1, G2, G3, G4, G1b, G2b, G3b, G4b = lts_map_graphs(G_lts, all_lts, gdf_nodes)

In [ ]:
# Find independent/disconnected grpahs
# Use to find areas to fix in OSM and filter out from heatmap
# https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.weakly_connected_components.html#networkx.algorithms.components.weakly_connected_components

graphSizes = [len(c) for c in sorted(nx.weakly_connected_components(G4), key=len, reverse=True)]
isolates = list(nx.isolates(G4))

# largest_cc = max(nx.weakly_connected_components(G_lts), key=len)
# S = [G4.subgraph(c).copy() for c in nx.weakly_connected_components(G4)]
# S0 = S[0]

In [ ]:
alpha_shape = boundry_polygon(gdf_nodes, 200)
valid_points = grid_points(alpha_shape, 25)
plot_grid_boundary(gdf_nodes, alpha_shape, valid_points)

In [ ]:
travel_speed = 15 #biking speed in km/hour

G1b, G2b, G3b, G4b = edge_travel_times(travel_speed, G1b, G2b, G3b, G4b)

In [ ]:
trip_time = 15 # minutes

# point to start isochrone plot from
y = 42.373696
x = -71.110564

point, nodeID = nearest_node(x, y, G1)

node_colors, node_count = point_isochrone(nodeID, trip_time, G1b, G2b, G3b, G4b)

point_isochrone_plot(city, point, node_colors, trip_time, G4b)

In [ ]:
nodeIsochroneSummary = []
for i, pt in enumerate(valid_points):
    print(f'\nPoint {i} at {pt.y}, {pt.x}')
    point, nodeID = nearest_node(pt.x, pt.y, G1)
    _, nodeCount = point_isochrone(nodeID, trip_time, G1b, G2b, G3b, G4b)
    nodeIsochroneSummary.append([point, nodeID] + nodeCount)

nodeIsochroneSummarydf = pd.DataFrame(nodeIsochroneSummary, columns=['point_node', 'node_id', 'LTS4', 'LTS3', 'LTS2', 'LTS1'])
nodeIsochroneSummarydf['point_grid'] = valid_points

In [ ]:
# nodeIsochroneSummarydf.sort_values(by=['LTS4'], ascending=True).head(15)
nodeIsochroneSummarydf.sort_values(by=['LTS2'], ascending=False).head(15)

In [ ]:
gs = gpd.GeoSeries(nodeIsochroneSummarydf['point_node'])

nodeIsochroneSummaryGDF = gpd.GeoDataFrame(nodeIsochroneSummarydf, geometry=gs, crs='wgs84')

nis1 = gpd.GeoDataFrame(nodeIsochroneSummarydf['LTS1'], geometry=gs, crs='wgs84')
nis2 = gpd.GeoDataFrame(nodeIsochroneSummarydf['LTS2'], geometry=gs, crs='wgs84')
nis3 = gpd.GeoDataFrame(nodeIsochroneSummarydf['LTS3'], geometry=gs, crs='wgs84')
nis4 = gpd.GeoDataFrame(nodeIsochroneSummarydf['LTS4'], geometry=gs, crs='wgs84')

nodeIsochroneSummaryGDF

In [ ]:
 # Initialize plot
plt.figure(figsize=(15,9))
ax = plt.axes(projection=ccrs.PlateCarree())

 # Plot input points
gdf_proj = gdf_nodes.to_crs(ccrs.Robinson().proj4_init)
ax.scatter([p.x for p in gdf_proj['geometry']],
            [p.y for p in gdf_proj['geometry']],
            transform=ccrs.Robinson(),
            marker='.', s=1, alpha=0.2)

ax.scatter([p.x for p in valid_points],
            [p.y for p in valid_points],
          #   transform=ccrs.Robinson(),
            marker='x', s=10, c='k')

m = 3000

s4 = m * (nodeIsochroneSummarydf['LTS4'] / nodeIsochroneSummarydf['LTS4'].max())
ax.scatter([p.x for p in nodeIsochroneSummarydf['point_node']],
            [p.y for p in nodeIsochroneSummarydf['point_node']],
          #   transform=ccrs.Robinson(),
            marker='.', s=s4, c='grey', alpha=0.5)

s3 = m * (nodeIsochroneSummarydf['LTS3'] / nodeIsochroneSummarydf['LTS4'].max())
ax.scatter([p.x for p in nodeIsochroneSummarydf['point_node']],
            [p.y for p in nodeIsochroneSummarydf['point_node']],
          #   transform=ccrs.Robinson(),
            marker='.', s=s3, c='r', alpha=0.5)

s2 = m * (nodeIsochroneSummarydf['LTS2'] / nodeIsochroneSummarydf['LTS4'].max())
ax.scatter([p.x for p in nodeIsochroneSummarydf['point_node']],
            [p.y for p in nodeIsochroneSummarydf['point_node']],
          #   transform=ccrs.Robinson(),
            marker='.', s=s2, c='g', alpha=0.5)

 # Plot alpha shape
# ax.add_geometries(
#      alpha_shape.to_crs(ccrs.Robinson().proj4_init)['geometry'],
#      crs=ccrs.Robinson(), alpha=.2)

plt.show()

In [ ]:
lts = all_lts[all_lts['lts'] > 0]

layer_lts = lonboard.PathLayer.from_geopandas(
    gdf=lts[["geometry", "lts", "name"]], 
    width_scale=2
)
layer_lts.get_color = apply_categorical_cmap(
    values=lts["lts"],
    cmap={
        0: [0, 0, 0],  # black
        1: [0, 128, 0],  # green
        2: [0, 191, 255],  # blue
        3: [255, 165, 0],  # orange
        4: [255, 0, 0],  # red
    },
)

r = 100
norm = nodeIsochroneSummarydf['LTS4'].max()

layer_iso2 = lonboard.ScatterplotLayer.from_geopandas(
    nis2,
    get_fill_color=[0, 0, 255],
    get_radius = r * np.sqrt(nis2['LTS2'] / norm),
    opacity=0.2,
)

layer_iso3 = lonboard.ScatterplotLayer.from_geopandas(
    nis3,
    get_fill_color=[255, 0, 0],
    get_radius = r * np.sqrt(nis3['LTS3'] / norm),
    opacity=0.2,
)

layer_iso4 = lonboard.ScatterplotLayer.from_geopandas(
    nis4,
    get_fill_color=[150, 150, 150],
    get_radius = r * np.sqrt(nis4['LTS4'] / norm),
    opacity=0.2,
)


map = lonboard.Map(layers=[layer_lts, layer_iso4, layer_iso3, layer_iso2],
            #  basemap_style=lonboard.basemap.CartoBasemap.Positron,
                basemap_style=lonboard.basemap.CartoBasemap.DarkMatter,
                _height=700,
                )
map